# FASTQ Processing

> Processing FASTQ files for barcode extraction and classification

## Introduction
This notebook implements the FASTQ processing functionality for BarcodeSeqKit. It includes classes and functions for handling FASTQ files, extracting barcodes, and sorting reads into appropriate output files based on barcode detection.

In [ ]:
#| default_exp fastq_processing

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os
import re
import gzip
import logging
import shutil
from typing import List, Dict, Tuple, Set, Optional, Union, Iterator, Any
from pathlib import Path
from Bio.SeqIO.QualityIO import FastqGeneralIterator
import fnmatch
from BarcodeSeqKit.core import (
    BarcodeConfig, 
    BarcodeMatch, 
    OrientationType,
    BarcodeExtractor, 
    ExtractionStatistics,
    BarcodeLocationType
)
from BarcodeSeqKit.sequence_utils import classify_read_by_first_match

# FASTQ File Handler
First, let's define a class to handle FASTQ file operations:

In [ ]:
#| export
class FastqHandler:
    """Handles FASTQ file operations."""
    
    @staticmethod
    def find_fastq_pairs(directory: str) -> Tuple[str, str]:
        """Find paired FASTQ files in a directory.
        
        Args:
            directory: Directory to search for FASTQ files
            
        Returns:
            Tuple of (read1_path, read2_path)
            
        Raises:
            ValueError: If paired FASTQ files cannot be found
        """
        files = os.listdir(directory)
        # Common naming patterns for paired FASTQ files
        r1_patterns = ('_1.fq.gz', '_R1.fastq.gz', '_R1_001.fastq.gz')
        r2_patterns = ('_2.fq.gz', '_R2.fastq.gz', '_R2_001.fastq.gz')
        
        # Find files matching the patterns
        r1_file = None
        r2_file = None
        
        for f in files:
            if any(f.endswith(pattern) for pattern in r1_patterns):
                r1_file = f
            elif any(f.endswith(pattern) for pattern in r2_patterns):
                r2_file = f
        
        if not (r1_file and r2_file):
            raise ValueError(f"Could not find paired FASTQ files in {directory}")
            
        return os.path.join(directory, r1_file), os.path.join(directory, r2_file)
    
    @staticmethod
    def open_fastq(filepath: str) -> Union[gzip.GzipFile, 'TextIOWrapper']:
        """Open a FASTQ file, handling gzipped and non-gzipped files.
        
        Args:
            filepath: Path to the FASTQ file
            
        Returns:
            An opened file handle
        """
        if filepath.endswith(('.gz', '.gzip')):
            return gzip.open(filepath, "rt")
        else:
            return open(filepath, "rt")
    
    @staticmethod
    def fastq_pair_iterator(r1_path: str, r2_path: str) -> Iterator[Tuple[Tuple, Tuple]]:
        """Create an iterator over paired FASTQ reads.
        
        Args:
            r1_path: Path to the first read file
            r2_path: Path to the second read file
            
        Returns:
            Iterator yielding pairs of (read1, read2) tuples where each read is 
            a tuple of (title, sequence, quality)
        """
        with FastqHandler.open_fastq(r1_path) as f1, FastqHandler.open_fastq(r2_path) as f2:
            # Use the more efficient FastqGeneralIterator
            for read1, read2 in zip(FastqGeneralIterator(f1), FastqGeneralIterator(f2)):
                yield read1, read2

# FASTQ Output Manager
Now let's create a class to manage output FASTQ files

In [ ]:
#| export
class FastqOutputManager:
    """Manages output FASTQ files for different barcode categories."""
    
    def __init__(
        self, 
        output_prefix: str,
        output_dir: str,
        categories: List[str],
        compress: bool = True
    ):
        """Initialize the output manager.
        
        Args:
            output_prefix: Prefix for output files
            output_dir: Directory for output files
            categories: List of barcode categories (e.g., "barcode5_orientFR")
            compress: Whether to compress output files
        """
        self.output_prefix = output_prefix
        self.output_dir = output_dir
        self.categories = categories
        self.compress = compress
        self.file_handles = {}
        
        # Create output directory if it doesn't exist
        os.makedirs(output_dir, exist_ok=True)
        
        # Initialize file handles for each category
        self._init_file_handles()
    
    def _init_file_handles(self):
        """Initialize file handles for all categories."""
        for category in self.categories:
            self._open_category_files(category)
    
    def _open_category_files(self, category: str):
        """Open R1 and R2 files for a category.
        
        Args:
            category: Barcode category
        """
        r1_path = self._get_output_path(category, "R1")
        r2_path = self._get_output_path(category, "R2")
        
        mode = "wt" if self.compress else "w"
        
        if self.compress:
            self.file_handles[f"{category}_R1"] = gzip.open(r1_path, mode)
            self.file_handles[f"{category}_R2"] = gzip.open(r2_path, mode)
        else:
            self.file_handles[f"{category}_R1"] = open(r1_path, mode)
            self.file_handles[f"{category}_R2"] = open(r2_path, mode)
    
    def _get_output_path(self, category: str, read_suffix: str) -> str:
        """Generate an output file path based on category and read type.
        
        Args:
            category: Barcode category
            read_suffix: R1 or R2
            
        Returns:
            Path to the output file
        """
        if self.compress:
            extension = ".fastq.gz"
        else:
            extension = ".fastq"
            
        filename = f"{self.output_prefix}_{category}_{read_suffix}{extension}"
        return os.path.join(self.output_dir, filename)
    
    def write_read_pair(
        self, 
        category: str, 
        read1: Tuple[str, str, str], 
        read2: Tuple[str, str, str]
    ):
        """Write a read pair to the appropriate output files.
        
        Args:
            category: Barcode category
            read1: Read 1 tuple (title, sequence, quality)
            read2: Read 2 tuple (title, sequence, quality)
        """
        title1, seq1, qual1 = read1
        title2, seq2, qual2 = read2
        
        r1_handle = self.file_handles[f"{category}_R1"]
        r2_handle = self.file_handles[f"{category}_R2"]
        
        # Write in FastqGeneralIterator-compatible format
        r1_handle.write(f"@{title1}\n{seq1}\n+\n{qual1}\n")
        r2_handle.write(f"@{title2}\n{seq2}\n+\n{qual2}\n")
    
    def close(self):
        """Close all file handles."""
        for handle in self.file_handles.values():
            handle.close()
        
        self.file_handles = {}
    
    def __enter__(self):
        """Context manager enter method."""
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        """Context manager exit method."""
        self.close()

# FASTQ Extractor
Now let's implement the main FASTQ extractor class that inherits from the BarcodeExtractor base class:

In [ ]:
#| export
class FastqExtractor(BarcodeExtractor):
    """Extracts barcodes from FASTQ files."""
    
    def __init__(
        self, 
        barcodes: List[BarcodeConfig],
        output_prefix: str,
        fastq_files: List[str],
        output_dir: Optional[str] = None,
        merge_orientations: bool = False,
        keep_unmatched: bool = True,
        verbose: bool = False,
        log_file: Optional[str] = None,
        max_mismatches: int = 0,
        compress_output: bool = True,
        search_both_reads: bool = True
    ):
        """Initialize the FASTQ extractor.
        
        Args:
            barcodes: List of barcode configurations
            output_prefix: Prefix for output files
            fastq_files: List of FASTQ files (either 1 or 2 files)
            output_dir: Directory for output files
            merge_orientations: Whether to merge forward and reverse complement orientations
            keep_unmatched: Whether to keep reads without barcodes
            verbose: Whether to enable verbose logging
            log_file: Path to log file
            max_mismatches: Maximum number of mismatches allowed in barcode detection
            compress_output: Whether to compress output files
            search_both_reads: Whether to search for barcodes in both reads
        """
        super().__init__(
            barcodes=barcodes,
            output_prefix=output_prefix,
            output_dir=output_dir,
            merge_orientations=merge_orientations,
            keep_unmatched=keep_unmatched,
            verbose=verbose,
            log_file=log_file
        )
        
        # Validate FASTQ files
        if len(fastq_files) not in [1, 2]:
            raise ValueError("Either 1 or 2 FASTQ files must be provided")
        
        self.fastq_files = fastq_files
        self.max_mismatches = max_mismatches
        self.compress_output = compress_output
        self.search_both_reads = search_both_reads
        
        # Determine if we have paired-end data
        self.paired_end = len(fastq_files) == 2
        
        # Prepare output categories based on barcodes
        self.categories = self._prepare_categories()
        
        self.logger.info(f"Initialized FastqExtractor with {len(barcodes)} barcodes")
        self.logger.info(f"FASTQ files: {fastq_files}")
        self.logger.info(f"Output categories: {self.categories}")
    
    def _prepare_categories(self) -> List[str]:
        """Prepare output categories based on barcodes.
        
        Returns:
            List of category names
        """
        categories = []
        
        # Single barcode mode (no location specified) or multiple barcodes
        single_barcode_mode = all(b.location.value == "UNK" for b in self.barcodes)
        
        if single_barcode_mode:
            if self.merge_orientations:
                categories.append("barcode_orientAll")
            else:
                categories.extend(["barcode_orientFR", "barcode_orientRC"])
        else:
            # Multiple barcodes with locations
            for barcode in self.barcodes:
                location = barcode.location.value
                if location in ["5", "3"]:
                    if self.merge_orientations:
                        categories.append(f"barcode{location}_orientAll")
                    else:
                        categories.extend([
                            f"barcode{location}_orient{OrientationType.FORWARD.value}",
                            f"barcode{location}_orient{OrientationType.REVERSE_COMPLEMENT.value}"
                        ])
        
        # Add combined categories
        if len(self.barcodes) > 1 and not single_barcode_mode:
            categories.append("combined_barcodeAll")
        
        # Add no barcode category
        if self.keep_unmatched:
            categories.append("noBarcode")
        
        return categories
    
    def extract(self) -> ExtractionStatistics:
        """Extract barcodes from FASTQ files.
        
        Returns:
            Statistics from the extraction process
        """
        self.logger.info("Starting barcode extraction from FASTQ files")
        
        # Initialize statistics
        stats = ExtractionStatistics()
        
        # Initialize output files
        with FastqOutputManager(
            output_prefix=self.output_prefix,
            output_dir=self.output_dir,
            categories=self.categories,
            compress=self.compress_output
        ) as output_manager:
            
            # Process files
            if self.paired_end:
                self._process_paired_fastq(output_manager, stats)
            else:
                self._process_single_fastq(output_manager, stats)
        
        self.logger.info(f"Extraction complete: {stats.total_barcode_matches} matches in {stats.total_reads} reads")
        
        # Save statistics
        self.save_statistics()
        
        # Create combined files if requested
        if self.merge_orientations:
            self._create_merged_files()
        
        return stats
    
    def _process_paired_fastq(self, output_manager: FastqOutputManager, stats: ExtractionStatistics):
        """Process paired FASTQ files.
        
        Args:
            output_manager: Output file manager
            stats: Statistics object to update
        """
        r1_path, r2_path = self.fastq_files
        self.logger.info(f"Processing paired FASTQ files: {r1_path}, {r2_path}")
        
        read_count = 0
        
        for read1, read2 in FastqHandler.fastq_pair_iterator(r1_path, r2_path):
            read_count += 1
            stats.total_reads += 1
            
            # Update progress periodically
            if read_count % 100000 == 0 and self.verbose:
                self.logger.info(f"Processed {read_count} read pairs")
            
            # Extract title, sequence, and quality
            title1, seq1, qual1 = read1
            title2, seq2, qual2 = read2
            
            # Search for barcodes in read 1
            match1, category1 = classify_read_by_first_match(
                sequence=seq1,
                barcodes=self.barcodes,
                max_mismatches=self.max_mismatches
            )
            
            # If barcode not found in read 1 and we're searching both reads
            if category1 == "noBarcode" and self.search_both_reads:
                match2, category2 = classify_read_by_first_match(
                    sequence=seq2,
                    barcodes=self.barcodes,
                    max_mismatches=self.max_mismatches
                )
                
                if category2 != "noBarcode":
                    match = match2
                    category = category2
                else:
                    match = None
                    category = "noBarcode"
            else:
                match = match1
                category = category1
            
            # Update statistics
            if match:
                stats.update_barcode_match(match)
            else:
                stats.no_barcode_count += 1
            
            # Handle orientation merging
            if self.merge_orientations and match:
                barcode_location = match.barcode.location.value
                if barcode_location in ["5", "3"]:
                    category = f"barcode{barcode_location}_orientAll"
                else:
                    category = "barcode_orientAll"
            
            # Write to appropriate output file
            if match or self.keep_unmatched:
                output_manager.write_read_pair(category, read1, read2)
        
        self.logger.info(f"Finished processing {read_count} read pairs")
    
    def _process_single_fastq(self, output_manager: FastqOutputManager, stats: ExtractionStatistics):
        """Process a single FASTQ file as if it were paired (for testing).
        
        Args:
            output_manager: Output file manager
            stats: Statistics object to update
        """
        self.logger.warning("Processing single FASTQ file as paired (duplicating reads)")
        r1_path = self.fastq_files[0]
        
        read_count = 0
        
        with FastqHandler.open_fastq(r1_path) as f1:
            for title, seq, qual in FastqGeneralIterator(f1):
                read_count += 1
                stats.total_reads += 1
                
                # Update progress periodically
                if read_count % 100000 == 0 and self.verbose:
                    self.logger.info(f"Processed {read_count} reads")
                
                # Package as a read tuple
                read = (title, seq, qual)
                
                # Search for barcodes
                match, category = classify_read_by_first_match(
                    sequence=seq,
                    barcodes=self.barcodes,
                    max_mismatches=self.max_mismatches
                )
                
                # Update statistics
                if match:
                    stats.update_barcode_match(match)
                else:
                    stats.no_barcode_count += 1
                
                # Handle orientation merging
                if self.merge_orientations and match:
                    barcode_location = match.barcode.location.value
                    if barcode_location in ["5", "3"]:
                        category = f"barcode{barcode_location}_orientAll"
                    else:
                        category = "barcode_orientAll"
                
                # Write to appropriate output file (duplicating the read for both R1 and R2)
                if match or self.keep_unmatched:
                    output_manager.write_read_pair(category, read, read)
        
        self.logger.info(f"Finished processing {read_count} reads")
    
    def _create_merged_files(self):
        """Create merged files for combined categories."""
        self.logger.info("Creating merged files for combined categories")
        
        # Check if we need to create a combined file for all barcodes
        single_barcode_mode = all(b.location.value == "UNK" for b in self.barcodes)
        barcode_locations = set(b.location.value for b in self.barcodes if b.location.value in ["5", "3"])
        
        # Merge orientation files for each barcode location
        for location in barcode_locations:
            for read_suffix in ["R1", "R2"]:
                out_file = os.path.join(
                    self.output_dir, 
                    f"{self.output_prefix}_combined_barcode{location}_{read_suffix}.fastq"
                )
                
                # Find all files for this barcode location
                pattern = f"{self.output_prefix}_barcode{location}_orient*_{read_suffix}.fastq"
                if self.compress_output:
                    pattern += ".gz"
                    out_file += ".gz"
                
                input_files = []
                for f in os.listdir(self.output_dir):
                    if fnmatch.fnmatch(f, pattern):
                        input_files.append(os.path.join(self.output_dir, f))
                
                if input_files:
                    self._merge_fastq_files(input_files, out_file)
        
        # Create a combined file for all barcodes if needed
        if len(barcode_locations) > 1:
            for read_suffix in ["R1", "R2"]:
                out_file = os.path.join(
                    self.output_dir, 
                    f"{self.output_prefix}_combined_barcodeAll_{read_suffix}.fastq"
                )
                
                # Find all barcode files
                pattern = f"{self.output_prefix}_barcode*_orient*_{read_suffix}.fastq"
                if self.compress_output:
                    pattern += ".gz"
                    out_file += ".gz"
                
                input_files = []
                for f in os.listdir(self.output_dir):
                    if fnmatch.fnmatch(f, pattern):
                        input_files.append(os.path.join(self.output_dir, f))
                
                if input_files:
                    self._merge_fastq_files(input_files, out_file)
    
    def _merge_fastq_files(self, input_files: List[str], output_file: str):
        """Merge multiple FASTQ files into one.
        
        Args:
            input_files: List of input file paths
            output_file: Output file path
        """
        self.logger.info(f"Merging {len(input_files)} files into {output_file}")
        
        if not input_files:
            return
        
        # Check if files are compressed
        is_compressed = input_files[0].endswith('.gz')
        
        if is_compressed:
            with gzip.open(output_file, 'wb') as out_f:
                for in_file in input_files:
                    with gzip.open(in_file, 'rb') as in_f:
                        shutil.copyfileobj(in_f, out_f)
        else:
            with open(output_file, 'wb') as out_f:
                for in_file in input_files:
                    with open(in_file, 'rb') as in_f:
                        shutil.copyfileobj(in_f, out_f)
    
    def _find_barcode_matches(self, sequence: str) -> List[BarcodeMatch]:
        """Find barcode matches in a sequence.
        
        Note: This implementation uses classify_read_by_first_match instead of
        finding all matches for efficiency.
        
        Args:
            sequence: The sequence to search in
            
        Returns:
            List of BarcodeMatch objects
        """
        match, _ = classify_read_by_first_match(
            sequence=sequence,
            barcodes=self.barcodes,
            max_mismatches=self.max_mismatches
        )
        
        if match:
            return [match]
        else:
            return []

# Example Usage
Let's demonstrate how to use the FASTQ extractor

In [ ]:
# Example usage of the FastqExtractor

# Create barcode configurations
barcode_5prime = BarcodeConfig(
    sequence="ACGTACGT",
    location=BarcodeLocationType.FIVE_PRIME,
    name="5",
    description="5' barcode example"
)

barcode_3prime = BarcodeConfig(
    sequence="TGCATGCA",
    location=BarcodeLocationType.THREE_PRIME,
    name="3",
    description="3' barcode example"
)

# Create a test FASTQ file
import gzip
import tempfile

def create_test_fastq_files():
    """Create test FASTQ files for demonstration."""
    temp_dir = tempfile.mkdtemp()
    
    # Create R1 file with some barcoded reads
    r1_path = os.path.join(temp_dir, "test_R1.fastq.gz")
    with gzip.open(r1_path, 'wt') as f:
        # Read with 5' barcode
        f.write("@read1\n")
        f.write("ACGTACGTAAAAAAAAAAAAAAAA\n")
        f.write("+\n")
        f.write("ACGTACGTAAAAAAAAAAAAAAAA\n")
        
        # Read with 3' barcode
        f.write("@read2\n")
        f.write("AAAAAAAAAAAAAAAATGCATGCA\n")
        f.write("+\n")
        f.write("AAAAAAAAAAAAAAAATGCATGCA\n")
        
        # Read with no barcode
        f.write("@read3\n")
        f.write("AAAAAAAAAAAAAAAAAAAAAAAA\n")
        f.write("+\n")
        f.write("AAAAAAAAAAAAAAAAAAAAAAAA\n")
    
    # Create R2 file with some barcoded reads
    r2_path = os.path.join(temp_dir, "test_R2.fastq.gz")
    with gzip.open(r2_path, 'wt') as f:
        # Matching reads for R1
        f.write("@read1\n")
        f.write("TTTTTTTTTTTTTTTTTTTTTTTT\n")
        f.write("+\n")
        f.write("TTTTTTTTTTTTTTTTTTTTTTTT\n")
        
        f.write("@read2\n")
        f.write("TTTTTTTTTTTTTTTTTTTTTTTT\n")
        f.write("+\n")
        f.write("TTTTTTTTTTTTTTTTTTTTTTTT\n")
        
        f.write("@read3\n")
        f.write("TTTTTTTTTTTTTTTTTTTTTTTT\n")
        f.write("+\n")
        f.write("TTTTTTTTTTTTTTTTTTTTTTTT\n")
    
    return r1_path, r2_path, temp_dir

# Create the test files
r1_path, r2_path, temp_dir = create_test_fastq_files()

# Create an output directory
output_dir = os.path.join(temp_dir, "output")
os.makedirs(output_dir, exist_ok=True)

# Initialize the extractor
extractor = FastqExtractor(
    barcodes=[barcode_5prime, barcode_3prime],
    output_prefix="test",
    fastq_files=[r1_path, r2_path],
    output_dir=output_dir,
    verbose=True
)

# Run the extraction
stats = extractor.extract()

# Print the results
print(f"Total reads: {stats.total_reads}")
print(f"Barcode matches: {stats.total_barcode_matches}")
print(f"No barcode: {stats.no_barcode_count}")
print(f"Matches by barcode: {stats.matches_by_barcode}")
print(f"Matches by orientation: {stats.matches_by_orientation}")

# List the output files
print("\nOutput files:")
for f in os.listdir(output_dir):
    print(f"- {f}")

# Clean up temporary files
import shutil
shutil.rmtree(temp_dir)

2025-03-12 18:03:30,169 - BarcodeSeqKit - INFO - Initialized FastqExtractor with 2 barcodes
2025-03-12 18:03:30,169 - BarcodeSeqKit - INFO - Initialized FastqExtractor with 2 barcodes
2025-03-12 18:03:30,169 - BarcodeSeqKit - INFO - FASTQ files: ['/var/folders/r1/gf3p5gjs5j9gw_49cj98r8b00000gp/T/tmp8vltvt_v/test_R1.fastq.gz', '/var/folders/r1/gf3p5gjs5j9gw_49cj98r8b00000gp/T/tmp8vltvt_v/test_R2.fastq.gz']
2025-03-12 18:03:30,169 - BarcodeSeqKit - INFO - FASTQ files: ['/var/folders/r1/gf3p5gjs5j9gw_49cj98r8b00000gp/T/tmp8vltvt_v/test_R1.fastq.gz', '/var/folders/r1/gf3p5gjs5j9gw_49cj98r8b00000gp/T/tmp8vltvt_v/test_R2.fastq.gz']
2025-03-12 18:03:30,170 - BarcodeSeqKit - INFO - Output categories: ['barcode5_orientFR', 'barcode5_orientRC', 'barcode3_orientFR', 'barcode3_orientRC', 'combined_barcodeAll', 'noBarcode']
2025-03-12 18:03:30,170 - BarcodeSeqKit - INFO - Output categories: ['barcode5_orientFR', 'barcode5_orientRC', 'barcode3_orientFR', 'barcode3_orientRC', 'combined_barcodeAll', '

Total reads: 3
Barcode matches: 2
No barcode: 1
Matches by barcode: {'5': 1, '3': 1}
Matches by orientation: {'FR': 2}

Output files:
- test_barcode3_orientRC_R2.fastq.gz
- test_barcode3_orientFR_R1.fastq.gz
- test_extraction_stats.json
- test_combined_barcodeAll_R2.fastq.gz
- test_extraction_stats.tsv
- test_noBarcode_R2.fastq.gz
- test_barcode5_orientRC_R1.fastq.gz
- test_barcode5_orientFR_R2.fastq.gz
- test_combined_barcodeAll_R1.fastq.gz
- test_barcode3_orientFR_R2.fastq.gz
- test_barcode3_orientRC_R1.fastq.gz
- test_barcode5_orientFR_R1.fastq.gz
- test_noBarcode_R1.fastq.gz
- test_barcode5_orientRC_R2.fastq.gz


# Conclusion
This notebook implements the FASTQ processing functionality for BarcodeSeqKit, providing efficient extraction and classification of barcoded reads from FASTQ files. The implementation uses the efficient FastqGeneralIterator for parsing FASTQ files and provides flexibility for handling both single-end and paired-end data.

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()